## 데이터 load 및 확인

In [1]:
import numpy as np
import pandas as pd

emotion_est = pd.read_csv("../result/action_estimation.csv")
action_est = pd.read_csv('../result/emotion_estimation.csv')

In [2]:
emotion_est

,감정,에너지,회복,순환,정화
0,행복,0.324557,0.344684,0.257535,0.372669
1,우울,0.256301,0.539817,0.321080,0.273804
2,슬픔,0.310976,0.308342,0.262105,0.256712
3,열정,0.269738,0.488793,0.353732,0.360805
4,분노,0.322202,0.440319,0.408658,0.398532
5,두려움,0.295698,0.295809,0.393288,0.315617


In [3]:
action_est

,행동,에너지,회복,순환,정화
0,명상,0.359411,0.508459,0.375176,0.364463
1,산책,0.319155,0.540431,0.464981,0.550028
2,등산,0.330991,0.432612,0.464082,0.329396
3,샤워,0.241211,0.316949,0.346316,0.285009
4,수영,0.356064,0.529326,0.358547,0.247034
5,조깅,0.271695,0.426948,0.428360,0.251207
6,자전거 타기,0.322527,0.257293,0.358325,0.177498
7,요가,0.295578,0.374230,0.427756,0.368902
8,스트레칭,0.271916,0.366607,0.424326,0.443960
9,여행,0.305515,0.288080,0.262577,0.317648


In [4]:
import numpy as np
from numpy import dot
from numpy.linalg import norm
from scipy.stats import pearsonr


def pearson_similarity(a, b):
  return np.dot((a - np.mean(a)), (b - np.mean(b))) / ((np.linalg.norm(a - np.mean(a))) * (np.linalg.norm(b - np.mean(b))))

In [5]:
# 감정 -> 효능 <- 행동 시나리오
class emotion2item():
    def __init__(self ,action_df, emotion_df):
        self.action_df = action_df
        self.emothon_df = emotion_df
        self.effs = ['에너지', '회복', '순환', '정화']
             

    def sim_item_MAE(self, em_idx, topn):
        em_name , * em_est = self.emothon_df.iloc[em_idx].tolist()
        em_est = np.array(em_est)
        
        em_dict = {}
        for idx in range(0,4):
            em_dict[self.effs[idx]] = em_est[idx]
        em_dict = sorted(em_dict.items() , reverse=True ,key= lambda x:x[1])
        
        top_em_dict = em_dict[:topn]
        top_em_keys = ['행동']
        top_em_values = []
        for em , value in top_em_dict:
            top_em_keys.append(em)
            top_em_values.append(value)
        top_em_values = np.array(top_em_values)
        
        recommend_dict = {}
        for idx in self.action_df.index:
            act_name , *act_est = self.action_df[top_em_keys].loc[idx]
            act_est = np.array(act_est)
            sim = np.mean(np.abs(top_em_values - act_est))
            #print(f'{act_name}과 {em_name}의 유사도(MAE)는 {sim} ')
            recommend_dict[act_name] = sim    
        recommend_dict = sorted(recommend_dict.items() ,key= lambda x:x[1])
        print(f'\'{em_name}\'을 고양(또는 완화)하는 행동은 \'{recommend_dict[0][0]}\'\n')        
    
    def sim_item_MAE_flow(self, em_idx, topn):
        em_name , * em_est = self.emothon_df.iloc[em_idx].tolist()
        em_est = np.array(em_est)
        
        em_dict = {}
        for idx in range(0,4):
            em_dict[self.effs[idx]] = em_est[idx]
        em_dict = sorted(em_dict.items() , reverse=True ,key= lambda x:x[1])
        top_em_dict = em_dict[:topn]
        top_em_keys = ['행동']
        top_em_values = []
        for em , value in top_em_dict:
            top_em_keys.append(em)
            top_em_values.append(value)
        top_em_values = np.array(top_em_values)
        print(f'감정({em_name})에 대해서 값이 높은 {topn}개의 차원 {top_em_dict}\n')
        print(f'상위 {topn}개의 차원에 대한 행동 유사도(MAE) \n')
        
        recommend_dict = {}
        for idx in self.action_df.index:
            act_name , *act_est = self.action_df[top_em_keys].loc[idx]
            act_est = np.array(act_est)
            sim = np.mean(np.abs(top_em_values - act_est))
            print(f'{act_name}과 {em_name}의 유사도(MAE)는 {sim} ')
            recommend_dict[act_name] = sim    
        recommend_dict = sorted(recommend_dict.items() ,key= lambda x:x[1])
        print(f'\n\'{em_name}\'을 고양(또는 완화)하는 행동은 \'{recommend_dict[0][0]}\'')

## 시나리오 테스트 
Setting : 감정하나에 대해서 추천을 진행
1. 감정 차원에서 상위 N개를 선택받음
2. 선택한 차원에 대해서 각 행동들에 대해서 MAE로 유사도 계산
3. 각 행동과 차원 값들에 대한 유사도(계산)에 대해서 오름차순으로 정렬 후 가장 작은 값을 선택

In [6]:
# 내부 과정 시각화
XYZ = emotion2item(action_est, emotion_est)
XYZ.sim_item_MAE_flow(0,2)


감정(행복)에 대해서 값이 높은 2개의 차원 [('정화', 0.3726692140102386), ('회복', 0.3446836769580841)]

상위 2개의 차원에 대한 행동 유사도(MAE) 

명상과 행복의 유사도(MAE)는 0.08599056303501129 
산책과 행복의 유사도(MAE)는 0.18655300140380862 
등산과 행복의 유사도(MAE)는 0.06560073494911195 
샤워과 행복의 유사도(MAE)는 0.05769776552915576 
수영과 행복의 유사도(MAE)는 0.15513869225978852 
조깅과 행복의 유사도(MAE)는 0.1018631100654602 
자전거 타기과 행복의 유사도(MAE)는 0.1412807494401932 
요가과 행복의 유사도(MAE)는 0.016656553745269753 
스트레칭과 행복의 유사도(MAE)는 0.046606957912445096 
여행과 행복의 유사도(MAE)는 0.05581238269805905 

'행복'을 고양(또는 완화)하는 행동은 '요가'


## 감정 차원중 값이 가장 큰 1개의 값만을 이용해서 추천

In [7]:
for idx in emotion_est.index:
    XYZ.sim_item_MAE(idx,1)

'행복'을 고양(또는 완화)하는 행동은 '요가'

'우울'을 고양(또는 완화)하는 행동은 '산책'

'슬픔'을 고양(또는 완화)하는 행동은 '여행'

'열정'을 고양(또는 완화)하는 행동은 '명상'

'분노'을 고양(또는 완화)하는 행동은 '등산'

'두려움'을 고양(또는 완화)하는 행동은 '명상'



## 감정 차원중 값이 가장 큰 2개의 값만을 이용해서 추천

In [8]:
for idx in emotion_est.index:
    XYZ.sim_item_MAE(idx,2)

'행복'을 고양(또는 완화)하는 행동은 '요가'

'우울'을 고양(또는 완화)하는 행동은 '수영'

'슬픔'을 고양(또는 완화)하는 행동은 '여행'

'열정'을 고양(또는 완화)하는 행동은 '명상'

'분노'을 고양(또는 완화)하는 행동은 '조깅'

'두려움'을 고양(또는 완화)하는 행동은 '명상'



## 감정 차원중 값이 가장 큰 3개의 값만을 이용해서 추천

In [9]:
for idx in emotion_est.index:
    XYZ.sim_item_MAE(idx,3)

'행복'을 고양(또는 완화)하는 행동은 '요가'

'우울'을 고양(또는 완화)하는 행동은 '수영'

'슬픔'을 고양(또는 완화)하는 행동은 '여행'

'열정'을 고양(또는 완화)하는 행동은 '명상'

'분노'을 고양(또는 완화)하는 행동은 '요가'

'두려움'을 고양(또는 완화)하는 행동은 '샤워'



## 감정 차원중 값이 가장 큰 4개의 값만을 이용해서 추천

In [10]:
for idx in emotion_est.index:
    XYZ.sim_item_MAE(idx,4)

'행복'을 고양(또는 완화)하는 행동은 '여행'

'우울'을 고양(또는 완화)하는 행동은 '수영'

'슬픔'을 고양(또는 완화)하는 행동은 '여행'

'열정'을 고양(또는 완화)하는 행동은 '명상'

'분노'을 고양(또는 완화)하는 행동은 '등산'

'두려움'을 고양(또는 완화)하는 행동은 '여행'



### 감정 -> 효능 -> 행동에서는 상위 1개 또는 2개 사용

# 가장 큰 값 1 ~ 4 까지 flow 확인

In [11]:
print('차원 값중에서 최대값 1 사용\n')
for idx in emotion_est.index:
        XYZ.sim_item_MAE_flow(idx,1)
        print("\n")

print('차원 값중에서 최대값 2 사용\n')
for idx in emotion_est.index:
        XYZ.sim_item_MAE_flow(idx,2)
        print("\n")

print('차원 값중에서 최대값 3 사용\n')
for idx in emotion_est.index:
        XYZ.sim_item_MAE_flow(idx,3)
        print("\n")

print('차원 값중에서 최대값 4 사용\n')
for idx in emotion_est.index:
        XYZ.sim_item_MAE_flow(idx,4)
        print("\n")

차원 값중에서 최대값 1 사용

감정(행복)에 대해서 값이 높은 1개의 차원 [('정화', 0.3726692140102386)]

상위 1개의 차원에 대한 행동 유사도(MAE) 

명상과 행복의 유사도(MAE)는 0.008206135034561135 
산책과 행복의 유사도(MAE)는 0.17735885977745064 
등산과 행복의 유사도(MAE)는 0.04327334761619572 
샤워과 행복의 유사도(MAE)는 0.08766058385372161 
수영과 행복의 유사도(MAE)는 0.12563551068305973 
조깅과 행복의 유사도(MAE)는 0.1214621007442474 
자전거 타기과 행복의 유사도(MAE)는 0.19517113566398622 
요가과 행복의 유사도(MAE)는 0.003766977787017789 
스트레칭과 행복의 유사도(MAE)는 0.07129058837890628 
여행과 행복의 유사도(MAE)는 0.055020821094512895 

'행복'을 고양(또는 완화)하는 행동은 '요가'


감정(우울)에 대해서 값이 높은 1개의 차원 [('회복', 0.5398174941539764)]

상위 1개의 차원에 대한 행동 유사도(MAE) 

명상과 우울의 유사도(MAE)는 0.03135882616043084 
산책과 우울의 유사도(MAE)는 0.0006133258342743142 
등산과 우울의 유사도(MAE)는 0.10720569491386411 
샤워과 우울의 유사도(MAE)는 0.2228687644004822 
수영과 우울의 유사도(MAE)는 0.010491943359374978 
조깅과 우울의 유사도(MAE)는 0.1128696978092193 
자전거 타기과 우울의 유사도(MAE)는 0.2825241804122925 
요가과 우울의 유사도(MAE)는 0.16558768749237057 
스트레칭과 우울의 유사도(MAE)는 0.17321048974990838 
여행과 우울의 유사도(MAE)는 0.25173776149